# Toy problem for TLS solution
We want to solve here the identification problem with TLS in a very simple case: a network with 2 nodes, without any structural contraint.
However, we introduce noise in polar coordinates and we try and use our covariance matrix estimators in order to weight the noise.

In [ ]:
import pandas as pd
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
from scipy import sparse

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.insert(1, '..')

from src.simulation.noise import add_noise_in_polar_coordinates
from src.models.noise_transformation import naive_noise_covariance
from src.models.error_in_variable import TotalLeastSquares, SparseTotalLeastSquare
from src.models.regression import ComplexLasso
from src.identification.error_metrics import fro_error, error_metrics

In [ ]:
np.set_printoptions(precision=8)

In [ ]:
samples = 300
magnitude_sd = 0.01
phase_sd = 0.01

In [ ]:
np.random.seed(11)

y_bus = np.array([
    [1+1j, 0, 0, 0],
    [0, 2+1j, 0, 0],
    [2+1j, 0, 1+1j, 0],
    [0, 0, 0, 2+1j],
])
nodes = y_bus.shape[0]

real_voltages = np.random.normal(1, 0.1, (samples, nodes)) + 1j*np.random.normal(1, 0.1, (samples, nodes))
real_currents = real_voltages @ y_bus

voltages, currents = add_noise_in_polar_coordinates(real_currents, real_voltages, magnitude_sd, phase_sd)

voltage_noise = voltages - real_voltages
current_noise = currents - real_currents

# Standard TLS

In [ ]:
eiv = TotalLeastSquares()
eiv.fit(voltages, currents)
y_eiv = eiv.fitted_admittance_matrix
y_eiv

In [ ]:
error_metrics(y_bus, y_eiv)

# Standard LASSO

In [ ]:
lasso = ComplexLasso(y_bus, verbose=False, lambdas=np.logspace(-4, 0, 20))
lasso.fit(voltages, currents)
y_lasso = lasso.fitted_admittance_matrix
y_lasso

In [ ]:
error_metrics(y_bus, y_lasso)

In [ ]:
lasso.best_trial

# L1-regularized TLS

In [ ]:
tls_lambda = lasso.best_trial.hyperparameters['lambda']
sparse_eiv = SparseTotalLeastSquare(lambda_value=tls_lambda, abs_tol=10e-12, rel_tol=10e-12, solver=cp.GUROBI, max_iterations=50)
sparse_eiv.fit(voltages, currents)
y_sparse_eiv = sparse_eiv.fitted_admittance_matrix

In [ ]:
error_metrics(y_bus, y_sparse_eiv)

In [ ]:
def plot_eiv_optimization(eiv_model, y_bus):
    y_errors = pd.Series([fro_error(y_bus, i.fitted_parameters) for i in eiv_model.iterations])
    pd.Series(y_errors).plot(title='Fro error on Y')
    plt.show()
    targets = pd.Series([i.target_function for i in eiv_model.iterations])
    pd.Series(targets).plot(title='Target function')
    plt.show()

In [ ]:
plot_eiv_optimization(sparse_eiv, y_bus)

# L1-regularized TLS with estimated covariance matrix

In [ ]:
sigma_voltage = naive_noise_covariance(voltages, magnitude_sd, phase_sd)
sigma_current = naive_noise_covariance(currents, magnitude_sd, phase_sd)

inv_sigma_current = sparse.linalg.inv(sigma_current)
inv_sigma_voltage = sparse.linalg.inv(sigma_voltage)

In [ ]:
sparse_eiv_cov = SparseTotalLeastSquare(lambda_value=1000, abs_tol=10e-12, rel_tol=10e-12, solver=cp.GUROBI, max_iterations=50)
sparse_eiv_cov.fit(voltages, currents, inv_sigma_voltage, inv_sigma_current)
y_sparse_eiv_cov = sparse_eiv_cov.fitted_admittance_matrix

In [ ]:
error_metrics(y_bus, y_sparse_eiv_cov)

In [ ]:
plot_eiv_optimization(sparse_eiv_cov, y_bus)